Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать `recall@k` и `precision@k`.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

### Входные данные

Вам дается две выборки с пользовательскими сессиями - `id`-шниками просмотренных и `id`-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: `id` просмотренных товаров через `,` затем идёт `;` после чего следуют `id` купленных товаров (если такие имеются), разделённые запятой. Например, `1,2,3,4;` или `1,2,3,4;5,6`.

Гарантируется, что среди `id` купленных товаров все различные.

### Важно

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
- Если товар не встречался в обучающей выборке, его популярность равна `0`.
- Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
- Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и `k` в `recall@k` / `precision@k`.

In [27]:
from dataclasses import dataclass
from operator import truth
from pathlib import Path
from typing import Iterable, List, Tuple


@dataclass
class Session:
    viewed: List[str]
    bought: List[str]

        
def read_sessions(path: Path) -> Iterable[Session]:
    for line in path.open('rt'):
        viewed, bought = line.strip().split(';')
        yield Session(viewed=viewed.split(','), bought=list(filter(truth, bought.split(','))))

In [28]:
train_sessions = list(read_sessions(Path('coursera_sessions_train.txt')))
test_sessions = list(read_sessions(Path('coursera_sessions_test.txt')))

На обучении постройте частоты появления `id` в просмотренных и в купленных (`id` может несколько раз появляться в просмотренных, все появления надо учитывать)

In [12]:
from collections import Counter
from itertools import chain

viewed_frequencies = Counter(chain.from_iterable(_.viewed for _ in train_sessions))
bought_frequencies = Counter(chain.from_iterable(_.bought for _ in train_sessions))

Реализуйте два алгоритма рекомендаций:
- сортировка просмотренных `id` по популярности (частота появления в просмотренных),
- сортировка просмотренных `id` по покупаемости (частота появления в покупках).

In [38]:
def precision(recommended: List[str], bought: List[str], k: int) -> float:
    return len(set(recommended) & set(bought)) / k


def recall(recommended: List[str], bought: List[str], k: int) -> float:
    return len(set(recommended) & set(bought)) / len(bought)

In [41]:
from more_itertools import unique_everseen


def make_recommendations(sessions: Iterable[Session], frequencies: Counter, k: int) -> Iterable[Tuple[List[str], List[str]]]:
    for session in sessions:
        if session.bought:
            unique_sorted_viewed = sorted(unique_everseen(session.viewed), key=frequencies.__getitem__, reverse=True)
            yield unique_sorted_viewed[:k], session.bought

Для данных алгоритмов выпишите через пробел `AverageRecall@1`, `AveragePrecision@1`, `AverageRecall@5`, `AveragePrecision@5` на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет).

In [39]:
from typing import Callable


def average_metric(
    sessions: Iterable[Session],
    frequencies: Counter,
    k: int,
    metric: Callable[[List[str], List[str], int], float],
) -> float:
    recommendations = list(make_recommendations(sessions, frequencies, k))
    return sum(metric(recommended, bought, k) for recommended, bought in recommendations) / len(recommendations)

In [42]:
def write_answer(path: Path, sessions: Iterable[Session], frequencies: Counter):
    with path.open('wt') as fp:
        fp.write(
            f'{average_metric(sessions, frequencies, 1, recall):.2f} '
            f'{average_metric(sessions, frequencies, 1, precision):.2f} '
            f'{average_metric(sessions, frequencies, 5, recall):.2f} '
            f'{average_metric(sessions, frequencies, 5, precision):.2f}'
        )


write_answer(Path('views_popularity_train.txt'), train_sessions, viewed_frequencies)
write_answer(Path('purchases_popularity_train.txt'), train_sessions, bought_frequencies)
write_answer(Path('views_popularity_test.txt'), test_sessions, viewed_frequencies)
write_answer(Path('purchases_popularity_test.txt'), test_sessions, bought_frequencies)